In [81]:
# ライブラリの読み込み
import os
import scipy as sp
import numpy as np
import sklearn
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
# グラフをインラインで表示させる
%matplotlib inline

In [124]:
def preprocess_res(train_flg):
    # train set の場合の処理
    if train_flg:
        df=pd.read_csv('./train_set.csv')
        df.drop(df[df['SOURCE']=='Condominium'].index,inplace=True)
        df.drop(df[df['PRICE']>=40000000].index,inplace=True)
        df.drop(df[df['YR_RMDL']<=100].index,inplace=True)
        df_y = df['PRICE']
    else:
        df=pd.read_csv('./test_set.csv')
        df.drop(df[df['SOURCE']=='Condominium'].index,inplace=True)
    # 使う変数を定義
    cols = ['CENSUS_TRACT','YR_RMDL','ROOMS','AC','FIREPLACES','GBA']
    
    df_x = df.loc[:,cols]
    #欠損値　-> 'NA'　に変換
    #df_x.loc[df_x['ZIPCODE'].isnull()==True,'ZIPCODE']=df_x['ZIPCODE'].median()
    #df_x.loc[df_x['ROOMS'].isnull()==True,'ROOMS']=df_x['ROOMS'].median()
    #df_x.loc[df_x['GRADE'].isnull()==True,'GRADE']='Average'
    df_x.loc[df_x['YR_RMDL'].isnull()==True,'YR_RMDL']=df_x['YR_RMDL'].median()
    df_x.loc[df_x['GBA'].isnull()==True,'GBA']=df_x['GBA'].median()
    #df_x.drop(df_x.columns[np.isnan(df_x).any()], axis=1)
    # dummy
    df_x=pd.get_dummies(df_x)
    # train set の場合のoutput
    if train_flg:
        return df_x,df_y
    # test set の場合のoutput、kaggleへ提出するファイルの規定により、idも一緒に出力する
    else:
        return df_x,df['Id']

In [125]:
def preprocess_con(train_flg):
    # train set の場合の処理
    if train_flg:
        df=pd.read_csv('./train_set.csv')
        df.drop(df[df['SOURCE']=='Residential'].index,inplace=True)
        df.drop(df[df['PRICE']>=40000000].index,inplace=True)
        df.drop(df[df['YR_RMDL']<=100].index,inplace=True)
        df_y = df['PRICE']
    else:
        df=pd.read_csv('./test_set.csv')
        df.drop(df[df['SOURCE']=='Residential'].index,inplace=True)
    
    # 使う変数を定義
    cols = ['CENSUS_TRACT','ROOMS','AC','EYB','FIREPLACES','LIVING_GBA']
    
    df_x = df.loc[:,cols]
    #欠損値　-> 'NA'　に変換
    #df_x.loc[df_x['ZIPCODE'].isnull()==True,'ZIPCODE']=df_x['ZIPCODE'].median()
    #df_x.loc[df_x['ROOMS'].isnull()==True,'ROOMS']=df_x['ROOMS'].median()
    #df_x.loc[df_x['GBA'].isnull()==True,'GBA']=df_x['GBA'].median()
    #df_x.drop(df_x.columns[np.isnan(df_x).any()], axis=1)
    #df_x.loc[df_x['AYB'].isnull()==True,'AYB']=df_x['AYB'].median()
    df_x.loc[df_x['LIVING_GBA'].isnull()==True,'LIVING_GBA']=df_x['LIVING_GBA'].median()
    #df_x.loc[df_x['COMPLEX_NUMBER'].isnull()==True,'COMPLEX_NUMBER']=df_x['COMPLEX_NUMBER'].median() 
    # dummy
    df_x=pd.get_dummies(df_x)
    # train set の場合のoutput
    if train_flg:
        return df_x,df_y
    # test set の場合のoutput、kaggleへ提出するファイルの規定により、idも一緒に出力する
    else:
        return df_x,df['Id']

In [126]:
# train set RESIDENTIAL
clf_res =LinearRegression()
# 前処理の統合変数を使う
df_x,df_y = preprocess_res(train_flg=True)

from sklearn.metrics import mean_squared_error
def rmse(y_true, y_pred):
    return np.sqrt(mean_squared_error(y_true, y_pred))

# linear regressionで検証
clf_res.fit(df_x, df_y)
rmse(clf_res.predict(df_x),df_y)

C:\Users\world\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3248: DtypeWarning: Columns (17,18,19,20,21,22,23,32,33,34,36,42,44) have mixed types. Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


462674.70363457635

In [127]:
# train set CONDMINIUM
clf_con =LinearRegression()
# 前処理の統合変数を使う
df_x,df_y = preprocess_con(train_flg=True)

from sklearn.metrics import mean_squared_error
def rmse(y_true, y_pred):
    return np.sqrt(mean_squared_error(y_true, y_pred))

# linear regressionで検証
clf_con.fit(df_x, df_y)
rmse(clf_con.predict(df_x),df_y)

C:\Users\world\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3248: DtypeWarning: Columns (17,18,19,20,21,22,23,32,33,34,36,42,44) have mixed types. Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


664687.2665700568

In [128]:
# test set
# 前処理の統合変数を使う
df_test_x,df_test_id = preprocess_res(train_flg=False)
y_pred = clf_res.predict(df_test_x)

pred_df = pd.DataFrame(y_pred, index=df_test_id, columns=["PRICE"])
pred_df.to_csv('./output_res.csv', header=True, index_label='Id')

C:\Users\world\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3248: DtypeWarning: Columns (17,18,19,20,21,22,23,32,33,34,36,42,44) have mixed types. Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


In [129]:
# test set
# 前処理の統合変数を使う
df_test_x,df_test_id = preprocess_con(train_flg=False)
y_pred = clf_con.predict(df_test_x)

pred_df = pd.DataFrame(y_pred, index=df_test_id, columns=["PRICE"])
pred_df.to_csv('./output_con.csv', header=True, index_label='Id')

C:\Users\world\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3248: DtypeWarning: Columns (17,18,19,20,21,22,23,32,33,34,36,42,44) have mixed types. Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):
